In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from pathlib import Path
from scipy.ndimage import label
import pandas as pd
import h5py
import tqdm

In [2]:
def average_attentions(attention_list):    
    summed_att=np.zeros_like(np.load(attention_list[0]))
    count_non_zero= np.zeros_like(summed_att)
    for attention in attention_list:
        att=np.load(attention)
        att=np.nan_to_num(att)
        summed_att=summed_att+att
        count_non_zero=count_non_zero+(att>0).astype(np.uint32)
    return summed_att,summed_att/count_non_zero,count_non_zero

def get_image(image_name,slide_folder):
    slide_name,scene_nr=get_slide_name(image_name)
    slide=Image.open(slide_name)
    scene = slide.get_scene(scene_nr)
    wsi = wsi[..., ::-1]
    orig_size=np.uint32(np.array(size)/8)
    return wsi,orig_size


In [3]:


def get_high_attention_patch(attention_map,slide_folder,image_name,save_path):

    return 


def overlay_colormap_on_rgb(rgb_image, heatmap_image, image_name,save_path,high_att_coord, cmap='viridis', alpha=0.4):

    alpha_channel = np.ones((rgb_image.shape[0], rgb_image.shape[1]), dtype=np.uint8) * 255
    rgba_rgb_image = np.dstack((rgb_image, alpha_channel))
    cmap = plt.get_cmap(cmap)
    heatmap_colored = (cmap(heatmap_image) * 255).astype(np.uint8)
    rotated_heatmap_coloured = np.fliplr(np.rot90(heatmap_colored, k=3))
    resized_binary_mask = np.rot90(heatmap_image, k=3)
    resized_heatmap = Image.fromarray(rotated_heatmap_coloured).resize((rgb_image.shape[1],rgb_image.shape[0]), Image.BILINEAR)
    resized_binary_mask=Image.fromarray(resized_binary_mask).resize((rgb_image.shape[1],rgb_image.shape[0]), Image.BILINEAR)
    pil_rgb_image = Image.fromarray(rgba_rgb_image)
    overlaid_image = Image.blend(pil_rgb_image, resized_heatmap, alpha)
    resized_binary_mask_expanded=np.expand_dims(np.flip(resized_binary_mask,1),-1)
    overlaid_image=np.array(overlaid_image)
    zero_array = np.ones_like(overlaid_image)*255
    overlaid_image = np.where(resized_binary_mask_expanded, overlaid_image, zero_array)
    overlaid_image[:,:,3]=255
    overlaid_image=Image.fromarray(overlaid_image,mode='RGBA')
    if save_path is not None:
        overlaid_image.save(Path(save_path)/(image_name+".png"))
        resized_heatmap.save(Path(save_path)/(image_name+"_heat.png"))
        #overlaid_image.save(Path(save_path)/image_name.replace(".npy",".png"))
    return pil_rgb_image


In [34]:

def map_attentions(image_id,attention_values,feature_dir="/mnt/ceph_vol/UCEC/features",feature_size=256,orig_downscale=4):
    file= sorted(Path(feature_dir).glob(f"*{image_id}*.h5"))[0]
    all_ims=[]
    with h5py.File(file, 'r') as h5_file:
        # Assuming the data is stored in the root of the HDF5 file
        # Change 'dataset_name' to the actual name of your dataset # Adjust this if necessary
        coords,slide_sizes = h5_file["coords"],h5_file["slide_sizes"]
        # Return the first entry in the dataset
        all_coord_len=0
        scenes=list(np.unique([int(c[0]) for c in coords]))
        for i in scenes:
            filtered_coords=[c for c in list(coords) if c[0]==i]
            c_l=len(filtered_coords)
            im=np.zeros(np.uint64(slide_sizes[i]/orig_downscale))
            for coord,attention in zip(filtered_coords,attention_values[all_coord_len:all_coord_len+c_l]):
                y,x=int(coord[1]),int(coord[2])
                im[x:x+feature_size,y:y+feature_size]=attention
            all_coord_len+=c_l
            im=scale_negatives(im)
            all_ims.append(im)
            #all_ims.append(im/np.nanmax(np.abs(im)))
    return all_ims

In [33]:
import numpy as np

def scale_negatives(arr):
    # Make a copy to avoid modifying the original array
    result = arr.copy()

    # Identify negative values
    negative_values = arr < 0

    # Scale negative values to the range 0-1
    min_val = arr[negative_values].min()
    max_val = arr[negative_values].max()
    
    # Avoid division by zero if all negative values are the same
    if min_val != max_val:
        result[negative_values] = (arr[negative_values] - min_val) / (max_val - min_val)
    
    return result


In [ ]:
done=[]
attention_folder="results/5foldcv/MTF_nll_surv_a0.0_lr2e-05_5foldcv_gc16_concat/tcga_ucec_MTF_nll_surv_a0.0_lr2e-05_5foldcv_gc16_concat_s1/patch_risk"
save_folder="/mnt/ceph_vol/figure_vis_6/"
preview_dir="/mnt/ceph_vol/UCEC/preview"
#extraction_list=["21-083_K1"]
attention_files=sorted(Path(attention_folder).glob("*.npy"))

for attention_file in tqdm.tqdm(attention_files):
    image_id=attention_file.name.replace(".npy","")
    attention_values=np.load(attention_file)
    attentions_mapped=map_attentions(image_id,attention_values)
    previews=sorted(Path(preview_dir).glob(f"*{image_id}*.png"))
    for attention_mapped,preview in zip(attentions_mapped,previews):
        prev_im=np.array(Image.open(preview))
        overlay_colormap_on_rgb(prev_im,attention_mapped,image_id,save_folder,None)
    print(attention_file.name)



  1%|          | 1/96 [01:19<2:05:20, 79.16s/it]

TCGA-5B-A90C.npy


  2%|▏         | 2/96 [02:23<1:50:20, 70.43s/it]

TCGA-5S-A9Q8.npy


  3%|▎         | 3/96 [03:31<1:47:17, 69.22s/it]

TCGA-A5-A0GB.npy


: 